In [1]:
import pandas as pd
import random 
import transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel
import os
import torch
torch.cuda.device(1)

import inspect
import re
from sklearn.utils import shuffle

In [2]:
dataset_name = 'HADOOP'

MDL_LABEL_NUM = 89
JRA_LABEL_NUM = 142
ISLANDORA_LABEL_NUM = 67
INFRA_LABEL_NUM = 51
HIVE_LABEL_NUM = 65
HBASE_LABEL_NUM = 68
HADOOP_LABEL_NUM = 37
FCREPO_LABEL_NUM = 22
CONF_LABEL_NUM = 128
CB_LABEL_NUM = 64
CASSANDRA_LABEL_NUM = 15
BAM_LABEL_NUM = 96

In [3]:
class ML_Classification:

    labels_Num = {'MDL': MDL_LABEL_NUM, 
    'JRA': JRA_LABEL_NUM, 'ISLANDORA': ISLANDORA_LABEL_NUM, 
    'INFRA': INFRA_LABEL_NUM, 'HIVE': HIVE_LABEL_NUM, 'HBASE': HBASE_LABEL_NUM, 'HADOOP': HADOOP_LABEL_NUM, 'FCREPO': FCREPO_LABEL_NUM, 'CONF': CONF_LABEL_NUM,
     'CB': CB_LABEL_NUM, 'CASSANDRA': CASSANDRA_LABEL_NUM, 'BAM': BAM_LABEL_NUM
     }
    
    def __init__(self, dataset_name = dataset_name, labels_num = labels_Num[dataset_name]):
        self.nlp_model = {'bert': 'bert-base-cased', 'roberta': 'roberta-base', 'xlnet': 'xlnet-base-cased', 'distilbert': 'distilbert-base-cased', 'xlm': 'xlm-roberta-base', 'electra': 'google/electra-base-discriminator'}
        self.nlp_model_name = 'distilbert'
        self.dataset_name = dataset_name
        self.labels_num = labels_num

#       self.data_location = '/home/a22106/python_practice/component_classification/DeepSoft-C/Dataset/Data/{}.csv'.format(self.dataset_name)
        
        # 데이터 위치
        self.data_location_aug = 'D:\\OneDrive\\Deepsoft_C_classification\\HADOOP_aug_mul3.csv'
        self.data_location_ori = 'D:\\OneDrive\\Deepsoft_C_classification\\HADOOP.csv'

        # 데이터 변수 입력
        self.data = pd.read_csv(self.data_location_aug) # 증강 데이터
        self.data_ori = pd.read_csv(self.data_location_ori) # 원본 데이터
        self.eval_index = []


    def refine_origin_data(self):
        data = self.data_ori
        
        data_onehot = data.drop(columns = ['issuekey', 'title', 'description', 'component'])
        data_label = []
        for i in range(len(data_onehot)):
            data_label.append(list(data_onehot.iloc[i]))

        # make 'data' value
        data_text = pd.Series(list(data["title"] + ' ' + data['description']), index = data.index)

        #data = data.drop(columns = ['issuekey', 'title', 'description', 'component'])
        self.data_ori = pd.DataFrame(data = {'text': data_text, 'labels': data_label})

        # 오리지날 데이터를 train, eval데이터로 분리
        self.train_data_ori, self.eval_data_ori = train_test_split(self.data_ori, test_size = 0.3)
        self.eval_data = self.eval_data_ori
    

    # 불러온 정제된 데이터 one hot을 str에서 list로 바꾸는 작업
    def labels_to_int(self):
        data = self.data
        changeChar = ' [],'
        for i in range(len(data)):
            for chanChar in changeChar:
                data['labels'][i] = data['labels'][i].replace(chanChar, '')
            data['labels'][i] = list(data['labels'][i])
            data['labels'][i] = list(map(int, data['labels'][i]))
        
        # 증강데이터의 train_data에서 evaluation부분 제거
        
        eval_index_list = list(self.eval_data.index)
        
        for aug_num in range(4):
            iidf2 = [i + 6152* aug_num for i in eval_index_list]
            self.eval_index = self.eval_index + iidf2

            #if aug_num == 0:
            #    eval_index = self.eval_data.index.append(self.eval_data_ori.index +(6152*aug_num))
            #else:
            #    for i in self.eval_data.index:
            #        self.eval_data.index.append(self.eval_data_ori.index +(6152*aug_num)
#                eval_index = eval_index.append(self.eval_data_ori.index +(6152*aug_num))

        self.train_data = data.drop(self.eval_index)

        #print(self.train_data, self.eval_index.sort())

# train, evaluation 데이터로 나누기 // 사용 안함
    def split_data_eval(self):
        data = self.data
        train_data_ori, eval_data_ori = train_test_split(self.data_ori, test_size = 0.3, random_state = 32)

        eval_data_indexList = eval_data_ori.index.append(eval_data_ori.index * 2)
        data = data.drop(eval_data_indexList)
        
        self.train_data, self.eval_data = train_test_split(data, test_size = 0.3, random_state = 32)


# 모델 parameter 설정
    def set_model(self):
        self.model = MultiLabelClassificationModel(self.nlp_model_name, self.nlp_model[self.nlp_model_name], num_labels = self.labels_num, args = {'output_dir': 'D:/OneDrive/Deepsoft_C_classification/Outputs/HADOOP/{}/outputs20/'.format(self.nlp_model_name), 'overwrite_output_dir': False, 'num_train_epochs':40, 'gradient_accumulation_steps': 2, 'max_seq_length': 128, 'learning_rate': 5e-5})

    def train_model(self):
        self.model.train_model(self.train_data)
    
    def eval_model(self):
        self.result, self.model_outputs, self.wrong_predictions = self.model.eval_model(self.eval_data)
    
    def predict(self):
        self.preds, self.outputs = self.model.predict(self.test_data)

In [370]:
ml = ML_Classification()

In [371]:
ml.refine_origin_data()
ml.labels_to_int()

In [8]:
ml.data[:6152]

,text,labels
0,tool to mount ndfs on linux tool to mount ndfs...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,make Configuration an interface The Configurat...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,DF enhancement: performance and win XP support...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,Adding some uniformity/convenience to environm...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,bufferSize argument is ignored in FileSystem.c...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...
6147,Switch to v2 of the S3 List Objects API in S3A...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
6148,namenode connect time out in cluster with 65 m...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6149,"Eliminate needless uses of FileSystem.exists, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6150,Dispose of unnecessary SASL servers The IPC se...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [402]:
print(shuffle(ml.train_data))
print(ml.eval_data)

                                                    text  \
10735  Typo in com.getPasswordFromCredentialProviders...   
18962  add lib#getHomeDirectory() method Their standa...   
18325  Authenticate with Kerberos credentials when re...   
6204   nicer lot of errors for distcp The unformatted...   
7979   IPC doesn't want handle IOEs thrown behind soc...   
...                                                  ...   
3059   Potential deadlock in IPC This cycle (see atta...   
2945   Changes to support Kerberos with non Sun JVM (...   
21555  Fix issues faced by developers Harsh recently ...   
23004  incorrect lookup in remote.c There is some inc...   
3504   Some improvements to the mailing lists webpage...   

                                                  labels  
10735  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
18962  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
18325  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
6204   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [382]:
i = 4

print(ml.train_data['text'][i + 6152])
print(ml.train_data['text'][i])
#print(ml.train_data.index.sort_values())

bufferSize This is returned in FileSystem.create(File, boolean, overwrite) org.apache.hadoop.fs.FileSystem.create(File File, boolean overwrite, int bufferSize)  ignores the input parameter bufferSize. It passes further down the internal pipeline, which includes the buffer size, but not the parameter value. This works fine within the file system, since everything that calls create extracts buffer size from the same config.  the although is probably affected by changes, see   org.apache.hadoop.io.SequenceFile.Sorter.MergeQueue.MergeQueue(int size, String outName, int done)  The attached patch would fix some.
bufferSize argument is ignored in FileSystem.create(File, boolean, int) org.apache.hadoop.fs.FileSystem.create(File f, boolean overwrite, int bufferSize)  ignores the input parameter bufferSize. It passes further down the internal configuration, which includes the buffer size, but not the parameter value. This works fine within the file system, since everything that calls create extr